I tried different models. The last model was the one that I used.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()

In [2]:
import pandas as pd
import os

df = pd.read_csv('train_data.csv')

for _, row in df.iterrows():
  f = row['img_name']
  l = row['label']
  os.replace(f'train_images/{f}', f'train_images/{l}/{f}')

In [3]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Load training data
train_dir = 'train_images'
train_image_paths_0 = [os.path.join(train_dir, '0', file) for file in os.listdir(os.path.join(train_dir, '0'))]
train_image_paths_1 = [os.path.join(train_dir, '1', file) for file in os.listdir(os.path.join(train_dir, '1'))]
train_image_paths = train_image_paths_0 + train_image_paths_1
train_labels = [0]*len(train_image_paths_0) + [1]*len(train_image_paths_1)

# Load test data
test_dir = 'test_images'
test_image_paths = [os.path.join(test_dir, file) for file in os.listdir(test_dir)]
test_labels = [0]*len(test_image_paths)  # Dummy labels for test set


In [4]:
from torchvision import transforms

# Add data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([  # For validation/test data
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            # Conv Block 1
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 224x224 → 112x112
            
            # Conv Block 2
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 112x112 → 56x56
            
            # Conv Block 3
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 56x56 → 28x28
            
            # Adaptive pooling to handle varying sizes
            nn.AdaptiveAvgPool2d(1)  # 28x28 → 1x1
        )
        self.classifier = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [9]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
model = model.to(device)

In [10]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.1)


In [12]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define data paths
train_data_path = "train_images"
test_data_path = "test_images"

# Define transformations with augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),        # Random crop to 224x224
    transforms.RandomHorizontalFlip(),        # Random horizontal flip
    transforms.RandomRotation(10),            # Random rotation ±10 degrees
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([     # For validation/test data
    transforms.Resize(256),                   # Resize to 256x256
    transforms.CenterCrop(224),               # Center crop to 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load full training dataset
full_train_dataset = datasets.ImageFolder(
    root=train_data_path,
    transform=train_transform  # Initial transform for training data
)

# Split into train and validation (80-20 split)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Override validation set transform to remove augmentation
val_dataset.dataset.transform = val_test_transform

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

import os
from PIL import Image

class UnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [
            os.path.join(root_dir, f) 
            for f in os.listdir(root_dir) 
            if os.path.isfile(os.path.join(root_dir, f))
        ]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image  # Returns only image, no label

# Usage
test_dataset = UnlabeledDataset(test_data_path, transform=val_test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
print(next(model.parameters()).device)  # Should show 'cuda' or 'cpu'
print(inputs.device)  # Should match model's device

cuda:0
cpu


In [20]:
model = Net().to(device)  # <-- Add .to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [22]:
best_val_loss = float('inf')
patience = 3
trigger_times = 0

# Training loop
for epoch in range(50):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)  # <-- Move to device
        labels = labels.to(device)  # <-- Move to device
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)  # <-- Move to device first!
            labels = labels.to(device)  # <-- Move to device first!
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Rest of your epoch logging...
    
    val_loss /= len(val_loader)
    val_acc = correct / total
    print(f'Epoch {epoch+1}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}, Val Acc: {val_acc}')
    
    # Learning rate scheduling
    scheduler.step(val_loss)
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping!")
            break

Epoch 1, Train Loss: 0.36900084001430566, Val Loss: 0.25299355052951455, Val Acc: 0.9170068027210885
Epoch 2, Train Loss: 0.2615396257192976, Val Loss: 0.2310144881569389, Val Acc: 0.9215419501133787
Epoch 3, Train Loss: 0.2582839638588653, Val Loss: 0.2185787732132535, Val Acc: 0.919047619047619
Epoch 4, Train Loss: 0.23782014914963773, Val Loss: 0.21214205199393674, Val Acc: 0.9317460317460318
Epoch 5, Train Loss: 0.22805680138180437, Val Loss: 0.21865986911175045, Val Acc: 0.9321995464852608
Epoch 6, Train Loss: 0.22297475352019502, Val Loss: 0.188397992618274, Val Acc: 0.9365079365079365
Epoch 7, Train Loss: 0.22263558979983022, Val Loss: 0.1892873805448197, Val Acc: 0.9392290249433106
Epoch 8, Train Loss: 0.2096282381064974, Val Loss: 0.18478156655918862, Val Acc: 0.9396825396825397
Epoch 9, Train Loss: 0.20548889948649035, Val Loss: 0.18194430229672487, Val Acc: 0.9412698412698413
Epoch 10, Train Loss: 0.19905544971824501, Val Loss: 0.18863429261398487, Val Acc: 0.937868480725623

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)  # Output layer for binary classification

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for epoch in range(20):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss / 100}')
            running_loss = 0.0
print('Finished Training')


Epoch 1, Batch 100, Loss: 0.6900391781330109
Epoch 1, Batch 200, Loss: 0.6720368576049804
Epoch 1, Batch 300, Loss: 0.6413507962226868
Epoch 1, Batch 400, Loss: 0.6180286934971809
Epoch 1, Batch 500, Loss: 0.615151093006134
Epoch 1, Batch 600, Loss: 0.5814591497182846
Epoch 2, Batch 100, Loss: 0.5642892575263977
Epoch 2, Batch 200, Loss: 0.5462285140156746
Epoch 2, Batch 300, Loss: 0.48377217918634413
Epoch 2, Batch 400, Loss: 0.3915087629854679
Epoch 2, Batch 500, Loss: 0.32680975273251534
Epoch 2, Batch 600, Loss: 0.28059460908174516
Epoch 3, Batch 100, Loss: 0.24788808040320873
Epoch 3, Batch 200, Loss: 0.2328538653254509
Epoch 3, Batch 300, Loss: 0.23175353705883026
Epoch 3, Batch 400, Loss: 0.22736606072634458
Epoch 3, Batch 500, Loss: 0.23295555870980025
Epoch 3, Batch 600, Loss: 0.22020757231861354
Epoch 4, Batch 100, Loss: 0.22321265410631896
Epoch 4, Batch 200, Loss: 0.2046079248934984
Epoch 4, Batch 300, Loss: 0.197715317979455
Epoch 4, Batch 400, Loss: 0.1885259116999805
Epo

In [16]:
torch.save(model._save_to_state_dict, "model.pth")

In [25]:
from PIL import Image
import os

class UnlabeledTestDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [
            os.path.join(root_dir, f) 
            for f in os.listdir(root_dir) 
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(img_path)  # Return image + filename

# Use validation transforms (no augmentation)
test_dataset = UnlabeledTestDataset(
    root_dir='test_images',
    transform=val_test_transform  # From previous code
)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Make predictions
# Load the best saved model
model = Net().to(device)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Test loop
predictions = []
filenames = []

with torch.no_grad():
    for images, batch_filenames in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        filenames.extend(batch_filenames)


# Save predictions to a CSV file
submission_df = pd.DataFrame({'img_name': os.listdir(test_dir), 'prediction': predictions})
submission_df.to_csv('predictions.csv', index=False)


: 

In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

# Load training data
train_dir = 'train_images'
train_image_paths_0 = [os.path.join(train_dir, '0', file) for file in os.listdir(os.path.join(train_dir, '0'))]
train_image_paths_1 = [os.path.join(train_dir, '1', file) for file in os.listdir(os.path.join(train_dir, '1'))]
train_image_paths = train_image_paths_0 + train_image_paths_1
train_labels = [0]*len(train_image_paths_0) + [1]*len(train_image_paths_1)

# Load test data
test_dir = 'test_images'
test_image_paths = [os.path.join(test_dir, file) for file in os.listdir(test_dir)]
test_labels = [0]*len(test_image_paths)  # Dummy labels for test set

# Define data augmentation transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.RandomHorizontalFlip(),  # Flip horizontally
    transforms.RandomRotation(30),  # Rotate up to 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Adjust brightness, contrast, saturation
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

# Create datasets and data loaders
train_dataset = CustomDataset(train_image_paths, train_labels, transform=transform)
test_dataset = CustomDataset(test_image_paths, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [5]:


# Define a model with dropout and different activation functions
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.AvgPool2d(2, 2)  # Use average pooling
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        self.activation = nn.LeakyReLU()  # Use LeakyReLU activation

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer with L2 regularization
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01)  # L2 regularization
model = model.to(device)
# Define a function to calculate accuracy
def calculate_accuracy(model, loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Initialize lists to track metrics
train_loss_history = []
train_accuracy_history = []
test_loss_history = []
test_accuracy_history = []

# Initialize early stopping parameters
patience = 5  # Number of epochs to wait before stopping
best_test_loss = float('inf')  # Initialize best test loss
epochs_without_improvement = 0

# Train the model
# Train the model
for epoch in range(30):  
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss / 100}')
            running_loss = 0.0
    




Epoch 1, Batch 100, Loss: 0.6889347892999649
Epoch 1, Batch 200, Loss: 0.6806009197235108
Epoch 1, Batch 300, Loss: 0.6665885490179062
Epoch 1, Batch 400, Loss: 0.6533500301837921
Epoch 1, Batch 500, Loss: 0.648356072306633
Epoch 1, Batch 600, Loss: 0.6315322688221932
Epoch 1, Batch 700, Loss: 0.6344472920894623
Epoch 1, Batch 800, Loss: 0.630073545575142
Epoch 2, Batch 100, Loss: 0.6146052217483521
Epoch 2, Batch 200, Loss: 0.5864030507206917
Epoch 2, Batch 300, Loss: 0.5861467915773392
Epoch 2, Batch 400, Loss: 0.5605809128284455
Epoch 2, Batch 500, Loss: 0.5435501608252525
Epoch 2, Batch 600, Loss: 0.49788765370845794
Epoch 2, Batch 700, Loss: 0.4417063404619694
Epoch 2, Batch 800, Loss: 0.37341064870357515
Epoch 3, Batch 100, Loss: 0.32830569058656695
Epoch 3, Batch 200, Loss: 0.31809785552322867
Epoch 3, Batch 300, Loss: 0.2801007092744112
Epoch 3, Batch 400, Loss: 0.2978315446525812
Epoch 3, Batch 500, Loss: 0.2895161090046167
Epoch 3, Batch 600, Loss: 0.2665874893218279
Epoch 3,

In [3]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Correct in_channels to 3 for RGB input
        self.conv = nn.Conv2d(3, 32, 3)  # Changed from 6 to 3
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 26 * 26, 512)
        self.fc2 = nn.Linear(512, 300)
        self.fc3 = nn.Linear(300, 120)
        self.fc4 = nn.Linear(120, 2)
        self.activation = nn.LeakyReLU()  # Use LeakyReLU activation


    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 26 * 26)
        x = F.relu(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
model = Net()
# Replace BCELoss with CrossEntropyLoss
criterion = nn.CrossEntropyLoss()  # Instead of nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01)  # L2 regularization
model = model.to(device)

NameError: name 'device' is not defined

In [54]:
def calculate_accuracy(model, loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

# Initialize lists to track metrics
train_loss_history = []
train_accuracy_history = []
test_loss_history = []
test_accuracy_history = []

# Initialize early stopping parameters
patience = 5  # Number of epochs to wait before stopping
best_test_loss = float('inf')  # Initialize best test loss
epochs_without_improvement = 0

# Train the model
# Train the model
for epoch in range(30):  
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss / 100}')
            running_loss = 0.0

Epoch 1, Batch 100, Loss: 0.6927591812610626
Epoch 1, Batch 200, Loss: 0.6915788787603379
Epoch 1, Batch 300, Loss: 0.6909683918952942
Epoch 1, Batch 400, Loss: 0.6885539555549621
Epoch 1, Batch 500, Loss: 0.6868093043565751
Epoch 1, Batch 600, Loss: 0.6828656160831451
Epoch 1, Batch 700, Loss: 0.6756742423772812
Epoch 1, Batch 800, Loss: 0.6688040602207184
Epoch 2, Batch 100, Loss: 0.659545236825943
Epoch 2, Batch 200, Loss: 0.6558779150247573
Epoch 2, Batch 300, Loss: 0.6558793145418167
Epoch 2, Batch 400, Loss: 0.661699030995369
Epoch 2, Batch 500, Loss: 0.6406351405382157
Epoch 2, Batch 600, Loss: 0.6405650895833969
Epoch 2, Batch 700, Loss: 0.6387910962104797
Epoch 2, Batch 800, Loss: 0.6316255408525467
Epoch 3, Batch 100, Loss: 0.6254412907361985
Epoch 3, Batch 200, Loss: 0.619899353981018
Epoch 3, Batch 300, Loss: 0.6170816588401794
Epoch 3, Batch 400, Loss: 0.610457630455494
Epoch 3, Batch 500, Loss: 0.6095782920718193
Epoch 3, Batch 600, Loss: 0.6006646713614464
Epoch 3, Batch

In [55]:
predictions = []
with torch.no_grad():
    for data in test_loader:
        inputs, _ = data
        inputs = inputs.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

submission_df = pd.DataFrame({'img_name': os.listdir(test_dir), 'prediction': predictions})
submission_df.to_csv('predictions.csv', index=False)